# Project Report

## Introduction
In recent years, carbon capture and sequestration (CCS) has been recognized as a promising solution to mitigate climate change. This project focuses on the Pressure Swing Adsorption/Vacuum Swing Adsorption (PSA/VSA) process to capture CO2 from post-combustion power plant flue gas using Metal Organic Frameworks (MOFs) as sorbents.

## Methodology
The study utilizes optimized MOF structures (Mg-MOF-74, NOTT-300, UTSA-20, UTSA-80, ZIF-8) with calculated charges, employing simulation parameters such as Zeo++ volpo samples (10,000), number of Widom cycles (50,000), number of GCMC initialization cycles (500), number of GCMC production cycles (5,000), maximum distance between pressure points (0.6), and pressure range (0.2 to 10.2 bars) at 25°C.

In [26]:
import pyiast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler


## Question 1: Compute the density, accessible surface area (ASA), accessible probe-occupiable volume (POAV), and porosity

In [27]:
# Question 1: Compute density, ASA, POAV, and porosity
# Insert paths to your CSV files from the ZIP here (e.g., parse 'Density', 'POAV_A^3', 'POAV_Volume_fraction' from each CSV)
properties = {
    'Mg-MOF-74': {'density': 'insert data from Mg CO2.csv or Mg N2.csv', 'asa': 'insert ASA data if available', 'poav': 'insert data from Mg CO2.csv or Mg N2.csv', 'porosity': 'insert data from Mg CO2.csv or Mg N2.csv'},
    'NOTT-300': {'density': 'insert data from NOTT CO2.csv or NOTT N2.csv', 'asa': 'insert ASA data if available', 'poav': 'insert data from NOTT CO2.csv or NOTT N2.csv', 'porosity': 'insert data from NOTT CO2.csv or NOTT N2.csv'},
    'UTSA-20': {'density': 'insert data from utsa 20 CO2.csv or UTSA 20 N2.csv', 'asa': 'insert ASA data if available', 'poav': 'insert data from utsa 20 CO2.csv or UTSA 20 N2.csv', 'porosity': 'insert data from utsa 20 CO2.csv or UTSA 20 N2.csv'},
    'ZIF-8': {'density': 'insert data from ZIF CO2.csv or ZIF N2.csv', 'asa': 'insert ASA data if available', 'poav': 'insert data from ZIF CO2.csv or ZIF N2.csv', 'porosity': 'insert data from ZIF CO2.csv or ZIF N2.csv'},
    'UTSA-80': {'density': 'insert data from UTSA 80 CO2.csv or UTSA 80 N2.csv', 'asa': 'insert ASA data if available', 'poav': 'insert data from UTSA 80 CO2.csv or UTSA 80 N2.csv', 'porosity': 'insert data from UTSA 80 CO2.csv or UTSA 80 N2.csv'}
}

structures = ['Mg-MOF-74', 'NOTT-300', 'UTSA-20', 'ZIF-8', 'UTSA-80']
for struct in structures:
    print(f"{struct}: Density = {properties[struct]['density']} g/cm^3, ASA = {properties[struct]['asa']} A^2, POAV = {properties[struct]['poav']} A^3, Porosity = {properties[struct]['porosity']}")
# Note: If ASA is not in CSVs, run pore analysis with probe radius 1.525Å as per PDF

Mg-MOF-74: Density = insert data from Mg CO2.csv or Mg N2.csv g/cm^3, ASA = insert ASA data if available A^2, POAV = insert data from Mg CO2.csv or Mg N2.csv A^3, Porosity = insert data from Mg CO2.csv or Mg N2.csv
NOTT-300: Density = insert data from NOTT CO2.csv or NOTT N2.csv g/cm^3, ASA = insert ASA data if available A^2, POAV = insert data from NOTT CO2.csv or NOTT N2.csv A^3, Porosity = insert data from NOTT CO2.csv or NOTT N2.csv
UTSA-20: Density = insert data from utsa 20 CO2.csv or UTSA 20 N2.csv g/cm^3, ASA = insert ASA data if available A^2, POAV = insert data from utsa 20 CO2.csv or UTSA 20 N2.csv A^3, Porosity = insert data from utsa 20 CO2.csv or UTSA 20 N2.csv
ZIF-8: Density = insert data from ZIF CO2.csv or ZIF N2.csv g/cm^3, ASA = insert ASA data if available A^2, POAV = insert data from ZIF CO2.csv or ZIF N2.csv A^3, Porosity = insert data from ZIF CO2.csv or ZIF N2.csv
UTSA-80: Density = insert data from UTSA 80 CO2.csv or UTSA 80 N2.csv g/cm^3, ASA = insert ASA data

## Question 2: Compute the Henry coefficients for CO2 and N2 at 25°C, rank the structures based on their affinity for CO2

In [28]:
# Question 2: Compute Henry coefficients and rank
# Insert Henry coefficients from CSVs (e.g., 'henry_coefficient_average' key, convert units if needed)
henry_co2 = {
    'Mg-MOF-74': 'insert data from Mg CO2.csv',
    'NOTT-300': 'insert data from NOTT CO2.csv',
    'UTSA-20': 'insert data from utsa 20 CO2.csv',
    'ZIF-8': 'insert data from ZIF CO2.csv',
    'UTSA-80': 'insert data from UTSA 80 CO2.csv'
}
henry_n2 = {
    'Mg-MOF-74': 'insert data from Mg N2.csv',
    'NOTT-300': 'insert data from NOTT N2.csv',
    'UTSA-20': 'insert data from UTSA 20 N2.csv',
    'ZIF-8': 'insert data from ZIF N2.csv (or None if not porous)',
    'UTSA-80': 'insert data from UTSA 80 N2.csv'
}

henry_co2_ranking = sorted(henry_co2.items(), key=lambda x: x[1], reverse=True)
print("Ranking based on CO2 Henry coefficients:", [x[0] for x in henry_co2_ranking])

Ranking based on CO2 Henry coefficients: ['UTSA-20', 'ZIF-8', 'UTSA-80', 'NOTT-300', 'Mg-MOF-74']


## Question 3: Compute CO2 and N2 pure component isotherms at 25°C for the given structures, plot the pure isotherms

In [ ]:
# Question 3: Plot pure isotherms
# Load isotherms from CSVs (parse 'isotherm' key, which is a JSON string with 'pressure' and 'loading_absolute_average')
# Example: df_mg_co2 = pd.read_json('insert path to parsed isotherm from Mg CO2.csv')
df_mg_co2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From Mg CO2.csv isotherm
df_nott_co2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From NOTT CO2.csv isotherm
df_utsa20_co2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From utsa 20 CO2.csv isotherm
df_zif_co2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From ZIF CO2.csv isotherm
df_utsa80_co2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From UTSA 80 CO2.csv isotherm

df_mg_n2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From Mg N2.csv isotherm
df_nott_n2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From NOTT N2.csv isotherm
df_utsa20_n2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From UTSA 20 N2.csv isotherm
df_utsa80_n2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From UTSA 80 N2.csv isotherm
df_zif_n2 = pd.DataFrame({'pressure': 'insert data here', 'loading': 'insert data here'})  # From ZIF N2.csv (may be empty)

# Fit isotherms using pyiast as in example notebook
# Example: MG_CO2_IT = pyiast.InterpolatorIsotherm(df_mg_co2, loading_key="loading", pressure_key="pressure")
MG_CO2_IT = 'insert fitted isotherm for Mg-MOF-74 CO2'  # Fit here
NOTT_CO2_IT = 'insert fitted isotherm for NOTT-300 CO2'
UTSA20_CO2_IT = 'insert fitted isotherm for UTSA-20 CO2'
ZIF_CO2_IT = 'insert fitted isotherm for ZIF-8 CO2'
UTSA80_CO2_IT = 'insert fitted isotherm for UTSA-80 CO2'

MG_N2_IT = 'insert fitted isotherm for Mg-MOF-74 N2'
NOTT_N2_IT = 'insert fitted isotherm for NOTT-300 N2'
UTSA20_N2_IT = 'insert fitted isotherm for UTSA-20 N2'
ZIF_N2_IT = 'insert fitted isotherm for ZIF-8 N2'  # May be None
UTSA80_N2_IT = 'insert fitted isotherm for UTSA-80 N2'

# Plot
plt.figure()
for name, df in [('Mg-MOF-74 CO2', df_mg_co2), ('NOTT-300 CO2', df_nott_co2), ('UTSA-20 CO2', df_utsa20_co2), 
                 ('ZIF-8 CO2', df_zif_co2), ('UTSA-80 CO2', df_utsa80_co2), ('Mg-MOF-74 N2', df_mg_n2), 
                 ('NOTT-300 N2', df_nott_n2), ('UTSA-20 N2', df_utsa20_n2), ('UTSA-80 N2', df_utsa80_n2)]:
    if len(df) > 0:
        plt.plot(df['pressure'], df['loading'], label=name)
plt.xlabel('Pressure (bar)')
plt.ylabel('Loading (mol/kg)')
plt.legend()
plt.show()

## Question 4: Calculate the CO2 working capacity (WC) for the different structures

In [ ]:
# Question 4: Calculate CO2 working capacity
P_ads = 1.0  # Adsorption pressure in bar
P_des = 0.2  # Desorption pressure in bar
y_CO2 = 0.15
y_N2 = 0.85

# Use fitted isotherms from Question 3
# Example WC calculation as in example notebook: q_ads_MG = pyiast.iast([P_ads * y_CO2, P_ads * y_N2], [MG_CO2_IT, MG_N2_IT])[0]
q_ads_MG = 'insert calculation using MG_CO2_IT and MG_N2_IT'
q_des_MG = 'insert calculation using MG_CO2_IT and MG_N2_IT'
MG_WC = q_ads_MG - q_des_MG

q_ads_NOTT = 'insert calculation using NOTT_CO2_IT and NOTT_N2_IT'
q_des_NOTT = 'insert calculation using NOTT_CO2_IT and NOTT_N2_IT'
NOTT_WC = q_ads_NOTT - q_des_NOTT

q_ads_UTSA20 = 'insert calculation using UTSA20_CO2_IT and UTSA20_N2_IT'
q_des_UTSA20 = 'insert calculation using UTSA20_CO2_IT and UTSA20_N2_IT'
UTSA20_WC = q_ads_UTSA20 - q_des_UTSA20

q_ads_ZIF = 'insert calculation using ZIF_CO2_IT and ZIF_N2_IT'
q_des_ZIF = 'insert calculation using ZIF_CO2_IT and ZIF_N2_IT'
ZIF_WC = q_ads_ZIF - q_des_ZIF

q_ads_UTSA80 = 'insert calculation using UTSA80_CO2_IT and UTSA80_N2_IT'
q_des_UTSA80 = 'insert calculation using UTSA80_CO2_IT and UTSA80_N2_IT'
UTSA80_WC = q_ads_UTSA80 - q_des_UTSA80

wc_values = {'Mg-MOF-74': MG_WC, 'NOTT-300': NOTT_WC, 'UTSA-20': UTSA20_WC, 'ZIF-8': ZIF_WC, 'UTSA-80': UTSA80_WC}
print("Working Capacities (mol/kg):", wc_values)

## Question 5: Calculate the CO2/N2 selectivity (S) for the different structures

In [ ]:
# Question 5: Calculate CO2/N2 selectivity
# Use fitted isotherms from Question 3
# Example selectivity as in example notebook: S_ads_MG = (q_ads_MG[0] / q_ads_MG[1]) * (y_N2 / y_CO2)
S_ads_MG = 'insert calculation using q_ads_MG from Question 4'
S_ads_NOTT = 'insert calculation using q_ads_NOTT from Question 4'
S_ads_UTSA20 = 'insert calculation using q_ads_UTSA20 from Question 4'
S_ads_ZIF = 'insert calculation using q_ads_ZIF from Question 4 (or inf if no N2)'
S_ads_UTSA80 = 'insert calculation using q_ads_UTSA80 from Question 4'

s_values = {'Mg-MOF-74': S_ads_MG, 'NOTT-300': S_ads_NOTT, 'UTSA-20': S_ads_UTSA20, 'ZIF-8': S_ads_ZIF, 'UTSA-80': S_ads_UTSA80}
print("Selectivities:", s_values)

## Question 6: Rank the structures using a WC vs. S plot

In [ ]:
# Question 6: Rank structures using WC vs. S plot
# Use values from Questions 4 and 5
S_ads_tot = np.array([S_ads_MG, S_ads_NOTT, S_ads_UTSA20, S_ads_UTSA80, S_ads_ZIF])
WC_tot = np.array([MG_WC, NOTT_WC, UTSA20_WC, UTSA80_WC, ZIF_WC])

plt.scatter(S_ads_tot, WC_tot)
plt.text(S_ads_MG, MG_WC, "Mg-MOF-74", color="red", horizontalalignment='right')
plt.text(S_ads_NOTT, NOTT_WC, "NOTT-300", color="blue", horizontalalignment='right')
plt.text(S_ads_UTSA20, UTSA20_WC, "UTSA-20", color="green", horizontalalignment='left')
plt.text(S_ads_UTSA80, UTSA80_WC, "UTSA-80", color="cyan", horizontalalignment='left')
plt.text(S_ads_ZIF, ZIF_WC, "ZIF-8", color="black", horizontalalignment='left')

plt.xlabel("Selectivity")
plt.ylabel("Working Capacity (mol/kg)")
plt.show()

## Question 7: Is the screening model conclusive as to which material performs best in a PSA/VSA process for post-combustion CO2 capture? Discuss the level at which the screening was performed and come up with more complex key performance indicators

The screening model provides a preliminary ranking based on working capacity and selectivity, but it is not conclusive for determining the best material for PSA/VSA processes. The analysis is conducted at a basic level, using ideal adsorption isotherms and simplified conditions (25°C, fixed pressures). The lack of N2 adsorption data for ZIF-8 limits its selectivity assessment. More complex key performance indicators (KPIs) could include energy consumption during adsorption/desorption cycles, material stability under repeated use, cost-effectiveness, and kinetic performance. These factors should be integrated for a comprehensive evaluation.

## Question 8: Organization of the report
1. Introduction: Overview of CCS and PSA/VSA technology
2. Methodology: Simulation parameters and data analysis
3. Question 1: Density, ASA, POAV, porosity
4. Question 2: Henry coefficients and CO2 affinity ranking
5. Question 3: CO2 and N2 isotherms
6. Question 4: CO2 working capacity
7. Question 5: CO2/N2 selectivity
8. Question 6: WC vs. S ranking
9. Question 7: Discussion on screening model and advanced KPIs
10. Conclusion: Summary of findings